In [ ]:
#imports
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import os
import math as math
import glob
import torchvision.transforms.functional as TF
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from torch import Tensor
from torch.nn import functional as F
import random

In [ ]:
#https://github.com/nyoki-mtl/pytorch-segmentation/blob/master/src/models/scse.py
#Import SCSE module
class SCSEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.channel_excitation = nn.Sequential(nn.Linear(channel, int(channel // reduction)),
                                                nn.ReLU(inplace=True),
                                                nn.Linear(int(channel // reduction), channel))
        self.spatial_se = nn.Conv2d(channel, 1, kernel_size=1,
                                    stride=1, padding=0, bias=False)

    def forward(self, x):
        bahs, chs, _, _ = x.size()

        # Returns a new tensor with the same data as the self tensor but of a different size.
        chn_se = self.avg_pool(x).view(bahs, chs)
        chn_se = torch.sigmoid(self.channel_excitation(chn_se).view(bahs, chs, 1, 1))
        chn_se = torch.mul(x, chn_se)
        alpha =1
        spa_se = torch.sigmoid(self.spatial_se(x))
        spa_se = torch.mul(x, spa_se)
        return torch.add(chn_se, spa_se)
 

In [ ]:
# Define Dice Loss Function 
def dice_score(x, y):
    a,b,c,d, = x.size()
    d = torch.zeros(a)
    for i in range(a):
        lower = (x[i]+y[i]).sum()
        upper = (x[i]*y[i]).sum()
        d[i] = 1-2*(upper/lower)
    loss = d.sum()/a
    return loss

In [ ]:
##Edited version of the Resnet34
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Optional


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        ##################
        ####  Here we add the decoder layers ####
        ##################
        # Works only for resnet 34 (i think)
        # Input parameters not 100% sure
        self.conv1_1x1 = torch.nn.Conv2d(in_channels=64,out_channels=128,kernel_size=(1,1), stride=1,padding=0)
        self.conv2_1x1 = torch.nn.Conv2d(in_channels=64,out_channels=128,kernel_size=(1,1), stride=1,padding=0)
        self.conv3_1x1 = torch.nn.Conv2d(in_channels=128,out_channels=128,kernel_size=(1,1), stride=1,padding=0)
        self.conv4_1x1 = torch.nn.Conv2d(in_channels=256,out_channels=128,kernel_size=(1,1), stride=1,padding=0)
        self.conv5_1x1 = torch.nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(1,1), stride=1,padding=0)
        # use kaiming uniform or normal to initialize weights
        self.Tconv4 = torch.nn.ConvTranspose2d(in_channels=512, out_channels=128, kernel_size=(2,2), stride=2, padding=0)
        torch.nn.init.kaiming_normal_(self.Tconv4.weight)
        self.Tconv3 = torch.nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=(2,2), stride=2, padding=0) 
        torch.nn.init.kaiming_normal_(self.Tconv3.weight)
        self.Tconv2 = torch.nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=(2,2), stride=2, padding=0)
        torch.nn.init.kaiming_normal_(self.Tconv2.weight)
        self.Tconv1 = torch.nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=(2,2), stride=2, padding=0)
        torch.nn.init.kaiming_normal_(self.Tconv1.weight)
        self.SCSE1 = SCSEBlock(channel=256)
        self.SCSE2 = SCSEBlock(channel=256)
        self.SCSE3 = SCSEBlock(channel=256)
        self.SCSE4 = SCSEBlock(channel=256)
        self.bn_decoder1 = norm_layer(256)
        self.bn_decoder2 = norm_layer(256)
        self.bn_decoder3 = norm_layer(256)
        self.bn_decoder4 = norm_layer(256)
        self.Tconvfinal = torch.nn.ConvTranspose2d(in_channels=256, out_channels=1, kernel_size=(2,2), stride=2, padding=0)
        torch.nn.init.kaiming_normal_(self.Tconvfinal.weight)
        self.sigmoid=nn.Sigmoid()
        
        self.group1 = nn.Sequential(self.conv1,self.bn1,self.relu,self.maxpool,self.layer1,self.layer2)
        self.group2 = nn.Sequential(self.layer3, self.layer4)
        self.group3 = nn.Sequential(self.conv1_1x1,self.conv2_1x1,self.conv3_1x1,self.conv4_1x1,self.conv5_1x1,self.Tconv4,
                               self.Tconv3,self.Tconv2,self.Tconv1,self.SCSE1,self.SCSE2,self.SCSE3,self.SCSE4,self.bn_decoder1,self.bn_decoder2,self.bn_decoder3,self.bn_decoder4,self.Tconvfinal,self.sigmoid)
        
        #
        #
        #These are not needed now

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        # Resnet Encoder
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        self.cache1 = torch.clone(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        self.cache2 = torch.clone(x)
        x = self.layer2(x)
        self.cache3 = torch.clone(x)
        x = self.layer3(x)
        self.cache4 = torch.clone(x)
        x = self.layer4(x)
        # Decoder here
        x = self.conv5_1x1(x)
        x = self.Tconv4(x)
        self.cache4 = self.conv4_1x1(self.cache4)
        x = torch.cat((self.cache4, x),1)
        x = self.relu(x)
        x = self.bn_decoder4(x)
        x = self.SCSE4(x)
        
        x = self.Tconv3(x)
        self.cache3 = self.conv3_1x1(self.cache3)
        x = torch.cat((self.cache3, x),1)
        x = self.relu(x)
        x = self.bn_decoder3(x)
        x = self.SCSE3(x)

        x = self.Tconv2(x)
        self.cache2 = self.conv2_1x1(self.cache2)
        x = torch.cat((self.cache2, x),1)
        x = self.relu(x)
        x = self.bn_decoder2(x)
        x = self.SCSE2(x)

        x = self.Tconv1(x)
        self.cache1 = self.conv1_1x1(self.cache1)
        x = torch.cat((self.cache1, x),1)
        x = self.relu(x)
        x = self.bn_decoder1(x)
        x = self.SCSE1(x)
        x = self.Tconvfinal(x)
        x = self.sigmoid(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict, strict=False)
    return model


def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)


def resnet34(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet50(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet101(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress,
                   **kwargs)


def resnet152(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)


def resnext50_32x4d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def resnext101_32x8d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)


def wide_resnet50_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet50_2', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def wide_resnet101_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet101_2', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)

In [ ]:
model = resnet34(pretrained=True)
model.eval()

In [ ]:
#### This sections is for importing the data and validatio sets from local device to python.
# the data is normalized too.####

def list_images(image_dir, filename_expression='*.jpg'):
        filenames = glob.glob(os.path.join(image_dir, filename_expression))
        filenames = sorted(filenames) # important for cross-platform compatiblity
        print(f'Found {len(filenames)} image files in the directory "{image_dir}"')
        return filenames
def list_images2(image_dir, filename_expression='*.PNG'):
        filenames = glob.glob(os.path.join(image_dir, filename_expression))
        filenames = sorted(filenames) # important for cross-platform compatiblity
        print(f'Found {len(filenames)} image files in the directory "{image_dir}"')
        return filenames
    
def import_and_format_data(dataset,type_data):

    #reference image (all images should have same tensor size as this one)
    N = len(dataset)
    # define placeholder for tensor data
    if type_data == 'dataset':
        a,b,c = (transforms.ToTensor()(plt.imread(dataset[0]))).size()
        dataset2 = torch.zeros(N,a,b,c)
    if type_data == 'validationset':
        a,b,c = (transforms.ToTensor()(plt.imread(dataset[0]))).size()
        dataset2 = torch.zeros(N,b,c)
    
    
    #define forloop for importing data to tensors taking into account unwanted rotations and sizes
    for i in range(N):
        # read tensors to be able to check sizes
        T_data = transforms.ToTensor()(plt.imread(dataset[i])) 
        T_val =  transforms.ToTensor()(plt.imread(dataset[i])) 
        #perform checks to see if resizing or rotating is needed
        if b == T_data.size(1) and c==T_data.size(1):
            dataset2[i] = transforms.ToTensor()(plt.imread(dataset[i]))
        elif c == T_data.size(1) and b==T_data.size(2):
            if type_data == 'dataset':
                dataset2[i] = transforms.ToTensor()((transforms.ToPILImage()(T_data).convert("RGB")).rotate(90, expand=True))
            elif type_data == 'validationset':
                dataset2[i] = transforms.ToTensor()((transforms.ToPILImage()(T_data)).rotate(90, expand=True))       
        else: 
            if round(b/c)!=round((T_data.size(1))/(T_data.size(1))):
                print('ratio height width unequal','reference=(',b,',',c,') and target=()',T_data.size(1),',',T_data.size(1),')')
            dataset2[i]= TF.resize(T_data,[b,c]) 
    return dataset2
    

    
    # Select dataset and directory of dataset (unpack the cfd_image and seg_gt in CFD and testcrop and traincrop in CRACK500)
dataset_for_model = 'CFD'
Selected_data_directory = 'Data1\\CFD\\'


if dataset_for_model =='CFD':

    ##import data from local device. !!You need to set this to directory of your data!! ##
    IMAGE_DIR = Selected_data_directory + '\\cfd_image'
    IMAGE_DIR2 = Selected_data_directory + '\\seg_gt'

    ## This part turns the dataset and validationset in usable torch tensors in desired format (Number of images, channels, width, height) ##
    N_train = 72
    N_test = 46
    #put data in list and read the length N of the list
    dataset = list_images(IMAGE_DIR)
    validationset = list_images2(IMAGE_DIR2)
    N = len(dataset)
    if N!=len(validationset):
        print('dataset and validationset do not match in size. Dataset=',N,'and validationset=',len(validationset))
    dataset = import_and_format_data(dataset,'dataset')
    validationset = import_and_format_data(validationset,'validationset')

if dataset_for_model == "CRACK500":

    ##import data from local device. 
    IMAGE_DIR = Selected_data_directory+'\\traincrop' 
    IMAGE_DIR2 = Selected_data_directory+'\\testcrop'

    ## This part turns the dataset and validationset in usable torch tensors in desired format (Number of images, channels, width, height) ##
    N_train = 3792
    N_test = 2248
    #put data in list and read the length N of the list
    train_dataset = list_images(IMAGE_DIR)
    train_validationset = list_images2(IMAGE_DIR)
    test_dataset = list_images(IMAGE_DIR2)
    test_validationset = list_images2(IMAGE_DIR2)
    
    dataset = train_dataset + test_dataset # dataset is actually total 3020 mistake in paper so trainset =1812 and testset =1208
    validationset = train_validationset + test_validationset 
    
    N = len(dataset)
    if N!=len(validationset):
        print('dataset and validationset do not match in size. Dataset=',N,'and validationset=',len(validationset))
        
    dataset = import_and_format_data(dataset,'dataset')
    validationset = import_and_format_data(validationset,'validationset')

In [ ]:
#### A data sorter which randomly turns the dataset into a train dataset and a test dataset and accordingly 
# the train validationset and test validationset. The ratio images in the train test is approx 3:2 ####

def alt_Datasorter(dataset, validationset, N, N_train, N_test):
    count_train = 0
    count_test = 0
    a,b,c,d = dataset.size()
    e,g,h = validationset.size()
    f = 1
    train_dataset = torch.zeros(N_train,b,c,d)
    test_dataset = torch.zeros(N_test,b,c,d)
    train_validationset = torch.zeros(N_train,f,g,h)
    test_validationset = torch.zeros(N_test,f,g,h)
    
    for i in range(N):
        k = np.random.rand(1)
        if k < 0.5:
            if count_train < N_train:
                train_dataset[count_train,:] = dataset[i,:]
                train_validationset[count_train] = validationset[i]
                count_train += 1
            else:
                test_dataset[count_test] = dataset[i]
                test_validationset[count_test] = validationset[i]
                count_test += 1
        else:
            if count_test < N_test:
                test_dataset[count_test] = dataset[i]
                test_validationset[count_test] = validationset[i]
                count_test += 1
            else:
                train_dataset[count_train] = dataset[i]
                train_validationset[count_train] = validationset[i]
                count_train += 1
    return train_dataset, test_dataset, train_validationset, test_validationset

train_dataset, test_dataset, train_validationset, test_validationset = alt_Datasorter(dataset, validationset, N, N_train, N_test)

train_input = [train_dataset, train_validationset]
test_input = [test_dataset, test_validationset]

In [ ]:
#### function which preprocesses batches of the data- and validationsets. For each image within the batch random rotations
#, flips and brightness changes are applied ####
def Transforms(x_batch, y_batch, d):
    for i in range(len(x_batch)):
        # randomly rotate the data set and validation set in a single angle 
        k = random.random()*360
        x_batch[i] = TF.rotate(x_batch[i], k)
        y_batch[i] = TF.rotate(y_batch[i], k)
        # randomly flip the data set and validation set in the horizontal and vertical direction
        if random.random() > 0.5:
            x_batch[i] = TF.hflip(x_batch[i])
            y_batch[i] = TF.hflip(y_batch[i])
        if random.random() > 0.5:
            x_batch[i] = TF.vflip(x_batch[i])
            y_batch[i] = TF.vflip(y_batch[i]) 
        # randomly adjust the brightness of the images with a change of 0.05
        if random.random() > 0.5:
            x_batch[i] = TF.adjust_hue(x_batch[i],0.05)
        else:
            x_batch[i] = TF.adjust_hue(x_batch[i],-0.05)
        # randomly adjust the contrast of the images with a change of 0.05
        if random.random() > 0.5:
            x_batch[i] = TF.adjust_contrast(x_batch[i],1.05)
        else:
            x_batch[i] = TF.adjust_contrast(x_batch[i],0.95)
        y_batch[i] = torch.round(y_batch[i])
    x_batch = TF.center_crop(x_batch, [320,320])
    y_batch = TF.center_crop(y_batch, [320,320])
    x_batch = TF.resize(x_batch,d)
    y_batch = TF.resize(y_batch,d)
    
    return x_batch, y_batch

In [ ]:
## All algorithm for calculation the precision, recall and F1 value for the predicted value and the True value. ##
def precision_recall_f1(test_dataset, test_validationset):
    N_batches = len(test_dataset)
    biggest_batch_a, biggest_batch_b,biggest_batch_c,biggest_batch_d = test_dataset[0].shape
    smallest_batch_a, smallest_batch_b, smallest_batch_c, smallest_batch_d = test_dataset[-1].shape
    N = (N_batches-1)*biggest_batch_a + smallest_batch_a
    Pr = torch.zeros(N)
    Re = torch.zeros(N)
    F1 = torch.zeros(N)
    for k in range(N_batches):
        with torch.no_grad():
            x_batch = test_dataset[k]
            y_batch = test_validationset[k]

            y_pred = model(x_batch)
            loss = dice_score(y_pred, y_batch)
            
            a,b,c,d = y_batch.shape
            y_k = y_pred
            y_k2 = y_batch
            y_predict = y_pred.reshape(a,c*d)
            y_batches = y_batch.reshape(a,c*d)
    
            for i in range(a):
                n = (k*biggest_batch_a)+i
                TP,FP,FN,TN,NOT_SORTED = 0,0,0,0,0
                y_predict[i] = y_predict[i]-torch.min(y_predict[i])
                y_predict[i] = y_predict[i]/(torch.max(y_predict[i]))
                y_predict[i] = torch.round(y_predict[i])
                y_batches[i] = y_batches[i]-torch.min(y_batches[i])
                y_batches[i] = y_batches[i]/(torch.max(y_batches[i]))
                y_batches[i] = torch.round(y_batches[i])

                for j in range(c*d):
                    shape = 'diamond'
                    # True positives are pixels that are either diamond or square shape in distance 2 pixel of ground truth
                    if j>d*2 and j<c*d-2*d:
                        if y_predict[i,j]==1:
                            if  y_batches[i,j]==1:
                                TP +=1
                            elif j%d!=0 and y_batches[i,j-1]==1 :
                                TP +=1
                            elif j%d!=d-1 and y_batches[i,j+1]==1 :
                                TP +=1
                            elif j%d!=1 and y_batches[i,j-2]==1 :
                                TP +=1
                            elif j%d!=d-2 and y_batches[i,j+2]==1:
                                TP +=1
                            elif j-d>=0 and y_batches[i,j-d]==1:
                                TP +=1
                            elif j-d>=0 and j%d!=0 and y_batches[i,j-d-1]==1:
                                TP +=1
                            elif j-d>=0 and j%d!=d-1 and y_batches[i,j-d+1]==1:
                                TP +=1
                            elif j-d>=0 and j%d!=1 and shape=='square' and y_batches[i,j-d-2]==1:
                                TP +=1
                            elif j-d>=0 and j%d!=d-2 and shape=='square' and y_batches[i,j-d+2]==1:
                                TP +=1
                            elif j-2*d>=0 and y_batches[i,j-2*d]==1:
                                TP +=1
                            elif j-2*d>=0 and j%d!=0 and shape=='square' and y_batches[i,j-2*d-1]==1:
                                TP +=1
                            elif j-2*d>=0 and j%d!=d-1 and shape=='square' and y_batches[i,j-2*d+1]==1:
                                TP +=1
                            elif j-2*d>=0 and j%d!=1 and shape=='square' and y_batches[i,j-2*d-2]==1:
                                TP +=1
                            elif j-2*d>=0 and j%d!=d-2 and shape=='square' and y_batches[i,j-2*d+2]==1:
                                TP +=1
                            elif c*d-j>=d and y_batches[i,j+d]==1:
                                TP +=1
                            elif c*d-j>=d and j%d!=0 and y_batches[i,j+d-1]==1:
                                TP +=1
                            elif c*d-j>=d and j%d!=d-1 and y_batches[i,j+d+1]==1:
                                TP +=1
                            elif c*d-j>=d and j%d!=1 and shape=='square' and y_batches[i,j+d-2]==1:
                                TP +=1
                            elif c*d-j>=d and j%d!=d-2 and shape=='square' and y_batches[i,j+d+2]==1:
                                TP +=1
                            elif c*d-j>=2*d and y_batches[i,j+d]==1:
                                TP +=1
                            elif c*d-j>=2*d and j%d!=0 and shape=='square' and y_batches[i,j+2*d-1]==1:
                                TP +=1
                            elif c*d-j>=2*d and j%d!=d-1 and shape=='square' and y_batches[i,j+2*d+1]==1:
                                TP +=1
                            elif c*d-j>=2*d and j%d!=1 and shape=='square' and y_batches[i,j+2*d-2]==1:
                                TP +=1
                            elif c*d-j>=2*d and j%d!=d-2 and shape=='square' and y_batches[i,j+2*d+2]==1:
                                TP +=1
                            else:
                                FP +=1
                        elif y_predict[i,j]==0 and y_batches[i,j]==1:
                            FN += 1
                        elif y_predict[i,j]==0 and y_batches[i,j]==0:
                            TN += 1
                    else:
                        if y_predict[i,j]==1 and y_batches[i,j]==1:
                            TP += 1
                        elif y_predict[i,j]==1 and  y_batches[i,j]==0:
                            FP += 1
                        elif y_predict[i,j]==0 and y_batches[i,j]==1:
                            FN += 1
                        elif y_predict[i,j]==0 and y_batches[i,j]==0:
                            TN += 1
                if TP+FP+FN+TN != c*d:
                    print('Not all values are being evaluated check code',
                          'fraction of data not sorted is',NOT_SORTED/(c*d))
                if TP + FN > 0:
                    Pr[n] = TP/(TP+FP)
                if TP + FN > 0:
                    Re[n] = TP/(TP+FN)
                if  Pr[n] + Re[n] > 0:
                    F1[n] = (2*Pr[n]*Re[n])/(Pr[n]+Re[n])
                
                print('Precision of train set:', Pr[n])
                print('Recall of train set:', Re[n])
                print('F1 of train set:', F1[n])
                print('')
  
    Pr_final = torch.sum(Pr)/N
    Re_final = torch.sum(Re)/N
    F1_final = torch.sum(F1)/N
    
                
    return Pr_final, Re_final, F1_final

In [ ]:
## Alternate dataloader for creating batches for the training of the neural network ##
def alternate_dataloader(data, batchsize):
    data , validation = [data[0],data[1]]
    a,b,c,d = data.shape
    e,f,g,h = validation.shape
    batchesdata = torch.zeros(math.floor(a/batchsize),batchsize,b,c,d)
    batchesvalidation= torch.zeros(math.floor(a/batchsize),batchsize,f,g,h)
    batchesL = torch.zeros(a%batchsize,b,c,d)
    batchesLvalidation = torch.zeros(a%batchsize,f,g,h)
    LISTdata = []
    LISTvalidation = []
    LISTfinal = []
    for i in range(math.ceil(a/batchsize)):
        if a-i*batchsize > batchsize:
            for j in range(batchsize):
                d= data[j+i*batchsize]
                batchesdata[i,j]= d
                batchesvalidation[i,j]=validation[j+i*batchsize]

        else:
            for j in range(a%batchsize):
                batchesL[j]=data[j+i*batchsize]
                batchesLvalidation[j]=validation[j+i*batchsize]
    
    for i in range(math.floor(a/batchsize)):
        LISTdata.append(batchesdata[i])
        LISTvalidation.append(batchesvalidation[i])
    LISTdata.append(batchesL)
    LISTvalidation.append(batchesLvalidation)
    LISTfinal = [LISTdata, LISTvalidation]
    return LISTfinal

In [ ]:
# preparing pre-proccesed randomly batch data for training and testing
train_loader = alternate_dataloader(train_input, batchsize=5)
test_loader = alternate_dataloader(test_input, batchsize=5)

train_dataset = train_loader[0]
train_validationset = train_loader[1]
test_dataset = test_loader[0]
test_validationset = test_loader[1]

In [ ]:
# parameters for training loop (non hyper-parameters)
epochs = 45
#set progressing image sizes
size = [128,256,320] 
i = 0
# Define list to store losses and performances of each iteration
Pr = np.zeros(epochs)
Re = np.zeros(epochs)
F1 = np.zeros(epochs)
train_losses = []

In [ ]:
#prepare hyper-parameters for training
#Set default learning rates
learning_rate = 0.005
learning_rate_group1 = learning_rate*(1/9)
learning_rate_group2 = learning_rate*(1/3)
learning_rate_group3 = learning_rate*1
Beta_group1 = [0.9, 0.999]
Beta_group2 = [0.9, 0.999]
Beta_group3 =  [0.9, 0.999]
eps_group1 = 10**(-8)
eps_group2 = 10**(-8)
eps_group3 = 10**(-8)
# Set schedular and schedular parameters
#Inputs for schedular
base_lr= [0.000001*learning_rate_group1, 
          0.000001*learning_rate_group2, 
          0.000001*learning_rate_group3]
max_lr=[learning_rate_group1, 
          learning_rate_group2, 
          learning_rate_group3]
Total_number_of_batches_in_train=len(train_dataset)
Total_number_of_steps = epochs*Total_number_of_batches_in_train
steps_till_Turn_point_lr_max=np.ceil(Total_number_of_steps*0.4)
steps_till_End = epochs*Total_number_of_batches_in_train - steps_till_Turn_point_lr_max

# Custom Schedular for the Lr cycling
scaling = 0.0001
def customlinspace(delta_x,y1,y2, scaling):
    y1 = y1
    y2 = y2*scaling
    delta_y = y2-y1
    delta_x = delta_x
    a = (delta_y / delta_x)
    b = y1
    x= np.linspace(0,delta_x,delta_x+1)
    y= (a*x)+b
    return y

space1 = customlinspace(int(steps_till_Turn_point_lr_max),0.05, 1, 1)
space2 = customlinspace(int(steps_till_End+1),1, 0.05, scaling)

In [ ]:
#### Main file for training and testing algorithm
print('starting...')

# move the input and model to GPU for speed if available
def try_gpu():
    """
    If GPU is available, return torch.device as cuda:0; else return torch.device
    as cpu.
    """
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

# Initialize optimizer
optimizer = torch.optim.AdamW(model.group1.parameters(),learning_rate_group1)
optimizer.add_param_group({'params':model.group2.parameters(),'lr':learning_rate_group2})
optimizer.add_param_group({'params':model.group3.parameters(),'lr':learning_rate_group3})
print(optimizer)
print(learning_rate_group3)
# Try using gpu instead of cpu
device = try_gpu()

#Intitialize first layer to be off
for param in model.group1.parameters():
    param.requires_grad = False
T1=0
T2=1
lrs = []
for epoch in range(epochs):
    print('Epoch: {:.0f}'.format(epoch+1))
    #Unfreeze the first layer after 15 epocht
    if epoch == 15:
        for param in model.group1.parameters():
            param.requires_grad = True
            
    # Network in training mode and to device
    model.train()
    model.to(device)
       
    if epoch == 15:
        i = 1
    if epoch == 30:
        i = 2
    # Training loop 
    for j in range(len(train_dataset)):
        # Learning rate cycler:
        if int(epoch*Total_number_of_batches_in_train + j) < int(steps_till_Turn_point_lr_max):
            optimizer.param_groups[0]['lr'] = space1[T1]*learning_rate_group1
            optimizer.param_groups[1]['lr'] = space1[T1]*learning_rate_group2
            optimizer.param_groups[2]['lr'] = space1[T1]*learning_rate_group3
            T1=T1+1
        else:
            optimizer.param_groups[0]['lr'] = space2[T2]*learning_rate_group1
            optimizer.param_groups[1]['lr'] = space2[T2]*learning_rate_group2
            optimizer.param_groups[2]['lr'] = space2[T2]*learning_rate_group3
            T2=T2+1
            
        x_batch = torch.clone(train_dataset[j])
        y_batch = torch.clone(train_validationset[j])
        x_batch, y_batch = Transforms(x_batch,y_batch,size[i])
        
        group3 = optimizer.param_groups[2]
        lrs.append(group3["lr"])
        
        # Set to same device
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        # Set the gradients to zero
        optimizer.zero_grad()
        
        # Perform forward pass
        y_pred = model(x_batch)
#         Compute the loss
        loss =dice_score(y_pred, y_batch)
        print(loss)
        train_losses.append(loss)
        # Backward computation and update
        loss.backward()
        optimizer.step()
    
    model.eval()
    Pr2 = 0
    Re2 = 0 
    F12 = 0
    
    # test looop
    if epoch == 44:
        Pr_final, Re_final, F1_final = precision_recall_f1(test_dataset, test_validationset)
        print('Precision of test set:', Pr_final)
        print('Recall of test set:', Re_final)
        print('F1 of test set:', F1_final)
        print('')
plt.plot(lrs)